# Predicciones con RandomForest

A continuación voy a aplicar mi modelo para predecir el tipo de sonido en base a un dataset nuevo de sonidos.

Para ellos voy a realizar dos predicciones, una con mi modelo básico, y otra con un modelo sobre el que he aplicado un PCA para disminuir el número de features a 20.

Empiezo importando las librerías que voy a necesitar.

In [2]:
import numpy as np
import pandas as pd
import pandas.io

import re

import matplotlib.pyplot as plt

import librosa.display
import librosa

from glob import glob

import ffmpeg

import os

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from pydub import AudioSegment
from pydub.utils import make_chunks

import pickle

# Modelo Básico

Empiezo importando el modelo que he guardado anteriormente, para trabajar con él.

In [3]:
filename = 'finalized_model_fourier_64.sav'

loaded_model = pickle.load(open(filename, 'rb'))

Continúo descomponiendo los audios en segmentos de 2 segundos, y sacando todas sus features.

In [ ]:
def decompose_files(data_dir, audio_files):
    for j in range(len(audio_files)):
        myaudio = AudioSegment.from_file(data_dir + '{}'.format(os.listdir(data_dir)[j])) 
        chunk_length_ms = 2000
        chunks = make_chunks(myaudio, chunk_length_ms)

        for i, chunk in enumerate(chunks):
            chunk_name = "{}{}.wav".format(os.listdir(data_dir)[j], i)
            #print("exporting", chunk_name)
            chunk.export('./audios/predicciones_fourier_32/{}'.format(chunk_name), format="mp3")
            
decompose_files('./audios/predicciones/', glob('./audios/predicciones' + '/*'))

In [ ]:
def get_features(data_dir, audio_files):
    momentos = {'ducha': 'Ducha', 'cena': 'Cena', 'washing': 'Lavadora', 'vacuum': 'Aspiradora', 
                'shaver': 'Afeitadora', 'hairdryer': 'Secador_pelo', 'airconditioner': 'Aire_acondicionado', 
                'cellphone': 'Telefono', 'comp': 'Tecleo', 'silence': 'Silencio', 'dryer': 'Secadora', 
                'blender': 'Licuadora', 'doorbell': 'Timbre', 'alarm': 'Alarma', 'faucet': 'Grifo', 
                'microwave': 'Microondas'}
    features = []
    for i in range(len(audio_files)):
        y, sr = librosa.load(audio_files[i], sr=8000, mono=True)
        name = os.listdir(data_dir)[i]
        pattern = "[._][\w]+"
        name = re.sub(pattern, '', name)
        if name in momentos.keys():
            momento = momentos.get(name)
        else:
            momento = 'Otro'
        mfcc = np.ndarray.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20))
        scem = np.ndarray.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        scom = np.ndarray.mean(librosa.feature.spectral_contrast(S=np.abs(librosa.stft(y)), sr=sr, n_bands=4))
        srom = np.ndarray.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
        sbwm = np.ndarray.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
        tempo = librosa.beat.tempo(onset_envelope=librosa.onset.onset_strength(y=y, sr=sr, hop_length=512), sr=sr, hop_length=512)[0]
        rmse = np.ndarray.mean(librosa.feature.rms(y=y))
        D = np.abs(np.fft.fft(y, n=64)[:8000*2 // 2])
        features.append([mfcc, scem, scom, srom, sbwm, tempo, rmse, 
                         D[0], D[1], D[2], D[3], D[4], D[5], 
                         D[6], D[7], D[8], D[9], D[10], D[11], 
                         D[12], D[13], D[14], D[15], D[16], D[17], 
                         D[18], D[19], D[20], D[21], D[22], D[23],
                         D[24], D[25], D[26], D[27], D[28], D[29],
                         D[30], D[31], D[32], D[33], D[34], D[35],
                         D[36], D[37], D[38], D[39], D[40], D[41],
                         D[42], D[43], D[44], D[45], D[46], D[47],
                         D[48], D[49], D[50], D[51], D[52], D[53],
                         D[54], D[55], D[56], D[57], D[58], D[59],
                         D[60], D[61], D[62], D[63], momento])
    return pd.DataFrame(features, columns=['mfcc', 'scem','scom', 'srom','sbwm', 'tempo', 'rmse', 
                                           'Fourier1', 'Fourier2', 'Fourier3', 'Fourier4', 'Fourier5', 
                                           'Fourier6', 'Fourier7', 'Fourier8', 'Fourier9', 'Fourier10',
                                           'Fourier11', 'Fourier12', 'Fourier13', 'Fourier14', 'Fourier15',
                                           'Fourier16', 'Fourier17', 'Fourier18', 'Fourier19', 'Fourier20',
                                           'Fourier21', 'Fourier22', 'Fourier23', 'Fourier24', 'Fourier25',
                                           'Fourier26', 'Fourier27', 'Fourier28', 'Fourier29', 'Fourier30',
                                           'Fourier31', 'Fourier32', 'Fourier33', 'Fourier34', 'Fourier35', 
                                           'Fourier36', 'Fourier37', 'Fourier38', 'Fourier39', 'Fourier40',
                                           'Fourier41', 'Fourier42', 'Fourier43', 'Fourier44', 'Fourier45',
                                           'Fourier46', 'Fourier47', 'Fourier48', 'Fourier49', 'Fourier50',
                                           'Fourier51', 'Fourier52', 'Fourier53', 'Fourier54', 'Fourier55',
                                           'Fourier56', 'Fourier57', 'Fourier58', 'Fourier59', 'Fourier60',
                                           'Fourier61', 'Fourier62', 'Fourier63', 'Fourier64', 'momento'])

prueba = get_features('./audios/predicciones_fourier_32/', glob('./audios/predicciones_fourier_32' + '/*.wav'))

In [ ]:
prueba.to_csv('data_fourier_aux_64.csv', index=False)

In [4]:
prueba = pd.read_csv('data_fourier_aux_64.csv')
prueba = prueba.drop(['Fourier33', 'Fourier34', 'Fourier35', 'Fourier36', 'Fourier37', 'Fourier38', 
                      'Fourier39', 'Fourier40', 'Fourier41', 'Fourier42', 'Fourier43', 'Fourier44', 
                      'Fourier45', 'Fourier46', 'Fourier47', 'Fourier48', 'Fourier49', 'Fourier50', 
                      'Fourier51', 'Fourier52', 'Fourier53', 'Fourier54', 'Fourier55', 'Fourier56', 
                      'Fourier57', 'Fourier58', 'Fourier59', 'Fourier60', 'Fourier61', 'Fourier62', 
                      'Fourier63', 'Fourier64'], axis=1)
prueba.head()

,mfcc,scem,scom,srom,sbwm,tempo,rmse,Fourier1,Fourier2,Fourier3,...,Fourier24,Fourier25,Fourier26,Fourier27,Fourier28,Fourier29,Fourier30,Fourier31,Fourier32,momento
0,-14.789252,1307.712330,22.329178,2490.112305,941.869915,117.187500,0.029125,0.160970,0.079082,0.210319,...,0.045369,0.024227,0.000395,0.031113,0.055632,0.057702,0.049728,0.050520,0.016087,Lavadora
1,-2.996813,1181.177661,33.464148,3242.187500,1190.281966,120.000000,0.085731,0.144583,0.236493,1.176860,...,0.262270,0.212622,0.285958,0.884343,0.885720,0.252407,0.185584,0.046308,0.053887,Microondas
2,-2.807277,1409.495596,23.275813,2779.785156,1055.763228,133.928571,0.096703,0.139041,1.181226,0.423315,...,0.272895,0.211818,0.253776,0.263246,0.368740,0.408622,0.051935,0.118482,0.073620,Aspiradora
3,-16.530776,707.460976,23.874835,747.314453,440.637551,133.928571,0.017683,0.078369,0.047832,0.099244,...,0.021677,0.020253,0.016907,0.016557,0.015067,0.013532,0.011492,0.006960,0.001571,Timbre
4,-4.882842,1925.234669,19.538132,3061.645508,968.023202,133.928571,0.038148,0.117407,0.126795,0.064049,...,0.436363,0.457345,0.142331,0.055110,0.074392,0.325966,0.024409,0.012572,0.034940,Ducha


En este paso intermedio, dado que hay una pequeña cantidad de audios cuyas features son todas cero, los elimino para no ensuciar el modelo.

In [5]:
prueba = prueba[prueba['Fourier24'] != 0].reset_index(drop=True)
prueba.head()

,mfcc,scem,scom,srom,sbwm,tempo,rmse,Fourier1,Fourier2,Fourier3,...,Fourier24,Fourier25,Fourier26,Fourier27,Fourier28,Fourier29,Fourier30,Fourier31,Fourier32,momento
0,-14.789252,1307.712330,22.329178,2490.112305,941.869915,117.187500,0.029125,0.160970,0.079082,0.210319,...,0.045369,0.024227,0.000395,0.031113,0.055632,0.057702,0.049728,0.050520,0.016087,Lavadora
1,-2.996813,1181.177661,33.464148,3242.187500,1190.281966,120.000000,0.085731,0.144583,0.236493,1.176860,...,0.262270,0.212622,0.285958,0.884343,0.885720,0.252407,0.185584,0.046308,0.053887,Microondas
2,-2.807277,1409.495596,23.275813,2779.785156,1055.763228,133.928571,0.096703,0.139041,1.181226,0.423315,...,0.272895,0.211818,0.253776,0.263246,0.368740,0.408622,0.051935,0.118482,0.073620,Aspiradora
3,-16.530776,707.460976,23.874835,747.314453,440.637551,133.928571,0.017683,0.078369,0.047832,0.099244,...,0.021677,0.020253,0.016907,0.016557,0.015067,0.013532,0.011492,0.006960,0.001571,Timbre
4,-4.882842,1925.234669,19.538132,3061.645508,968.023202,133.928571,0.038148,0.117407,0.126795,0.064049,...,0.436363,0.457345,0.142331,0.055110,0.074392,0.325966,0.024409,0.012572,0.034940,Ducha


In [6]:
classes = ['Afeitadora', 'Aire_acondicionado', 'Alarma', 'Aspiradora', 'Cena', 'Ducha', 'Grifo', 'Lavadora', 
           'Licuadora', 'Microondas', 'Secador_pelo', 'Secadora', 'Silencio', 'Tecleo', 'Telefono', 'Timbre']

X_new = prueba.loc[:, prueba.columns != 'momento']
y_new = loaded_model.predict(X_new)
y_new2 = loaded_model.predict_proba(X_new)

reconocidos = 0
aciertos = 0
for i in range(len(X_new)):
    predicted_list = y_new[i].tolist()
    predicted_class = classes[y_new2[i].tolist().index(max(y_new2[i]))]
    predicted_probability = y_new2[i].tolist()[y_new2[i].tolist().index(max(y_new2[i]))]
    if max(y_new2[i]) > 0:
        if predicted_class == prueba.momento[i]:
            #print(str(i) + '-  {}: {:.2f}% ----> {}'.format(predicted_class, predicted_probability*100, prueba.momento[i]))
            aciertos += 1
            reconocidos += 1
        else:
            #print(str(i) + '-  {}: {:.2f}% ----> {}'.format(predicted_class, predicted_probability*100, prueba.momento[i]))
            reconocidos += 1


print('Porcentaje de sonidos reconocidos: {:.2f}%.'.format(reconocidos/(len(X_new))*100))
print('Porcentaje de aciertos: {:.2f}%.'.format(aciertos/(len(X_new))*100))
print('Porcentaje de aciertos una vez reconoce el sonido: {:.2f}%.'.format(aciertos/reconocidos*100))

Porcentaje de sonidos reconocidos: 100.00%.
Porcentaje de aciertos: 52.48%.
Porcentaje de aciertos una vez reconoce el sonido: 52.48%.


# Predicciones con el modelo PCA de 20 features

Hago lo mismo que antes pero con el nuevo modelo sobre el que hemos aplicado el PCA.

In [7]:
prueba = pd.read_csv('data_fourier_aux_64.csv')
prueba = prueba.drop(['Fourier33', 'Fourier34', 'Fourier35', 'Fourier36', 'Fourier37', 'Fourier38', 
                      'Fourier39', 'Fourier40', 'Fourier41', 'Fourier42', 'Fourier43', 'Fourier44', 
                      'Fourier45', 'Fourier46', 'Fourier47', 'Fourier48', 'Fourier49', 'Fourier50', 
                      'Fourier51', 'Fourier52', 'Fourier53', 'Fourier54', 'Fourier55', 'Fourier56', 
                      'Fourier57', 'Fourier58', 'Fourier59', 'Fourier60', 'Fourier61', 'Fourier62', 
                      'Fourier63', 'Fourier64'], axis=1)
prueba.head()

,mfcc,scem,scom,srom,sbwm,tempo,rmse,Fourier1,Fourier2,Fourier3,...,Fourier24,Fourier25,Fourier26,Fourier27,Fourier28,Fourier29,Fourier30,Fourier31,Fourier32,momento
0,-14.789252,1307.712330,22.329178,2490.112305,941.869915,117.187500,0.029125,0.160970,0.079082,0.210319,...,0.045369,0.024227,0.000395,0.031113,0.055632,0.057702,0.049728,0.050520,0.016087,Lavadora
1,-2.996813,1181.177661,33.464148,3242.187500,1190.281966,120.000000,0.085731,0.144583,0.236493,1.176860,...,0.262270,0.212622,0.285958,0.884343,0.885720,0.252407,0.185584,0.046308,0.053887,Microondas
2,-2.807277,1409.495596,23.275813,2779.785156,1055.763228,133.928571,0.096703,0.139041,1.181226,0.423315,...,0.272895,0.211818,0.253776,0.263246,0.368740,0.408622,0.051935,0.118482,0.073620,Aspiradora
3,-16.530776,707.460976,23.874835,747.314453,440.637551,133.928571,0.017683,0.078369,0.047832,0.099244,...,0.021677,0.020253,0.016907,0.016557,0.015067,0.013532,0.011492,0.006960,0.001571,Timbre
4,-4.882842,1925.234669,19.538132,3061.645508,968.023202,133.928571,0.038148,0.117407,0.126795,0.064049,...,0.436363,0.457345,0.142331,0.055110,0.074392,0.325966,0.024409,0.012572,0.034940,Ducha


In [8]:
filename = 'finalized_model_fourier_64_PCA.sav'

new_model = pickle.load(open(filename, 'rb'))

In [9]:
features = ['mfcc', 'scem','scom', 'srom','sbwm', 'tempo', 'rmse', 
            'Fourier1', 'Fourier2', 'Fourier3', 'Fourier4', 'Fourier5', 
            'Fourier6', 'Fourier7', 'Fourier8', 'Fourier9', 'Fourier10',
            'Fourier11', 'Fourier12', 'Fourier13', 'Fourier14', 'Fourier15',
            'Fourier16', 'Fourier17', 'Fourier18', 'Fourier19', 'Fourier20',
            'Fourier21', 'Fourier22', 'Fourier23', 'Fourier24', 'Fourier25',
            'Fourier26', 'Fourier27', 'Fourier28', 'Fourier29', 'Fourier30',
            'Fourier31', 'Fourier32']

x = prueba.loc[:, features].values
y = prueba.loc[:,['momento']].values

x = StandardScaler().fit_transform(x)

In [10]:
pca = PCA(n_components=20)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents, 
                           columns = ['principal component 1',
                                      'principal component 2',
                                      'principal component 3',
                                      'principal component 4',
                                      'principal component 5',
                                      'principal component 6',
                                      'principal component 7',
                                      'principal component 8',
                                      'principal component 9',
                                      'principal component 10',
                                      'principal component 11',
                                      'principal component 12',
                                      'principal component 13',
                                      'principal component 14',
                                      'principal component 15',
                                      'principal component 16',
                                      'principal component 17',
                                      'principal component 18',
                                      'principal component 19',
                                      'principal component 20'])

principalDf.head()

,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,principal component 6,principal component 7,principal component 8,principal component 9,principal component 10,principal component 11,principal component 12,principal component 13,principal component 14,principal component 15,principal component 16,principal component 17,principal component 18,principal component 19,principal component 20
0,-2.337290,-0.498829,-0.858843,-0.518616,-0.060498,-0.192489,0.085027,-0.087409,0.579366,-0.001253,-0.191453,0.273827,-0.064134,0.188470,0.165769,0.762987,-0.111273,-0.262081,0.053024,-0.253382
1,1.689775,-0.514729,-0.971589,-2.992900,1.103809,2.524404,0.129749,4.122140,1.998267,-0.305412,-0.040593,1.369755,-0.638422,-0.502953,-0.906961,-1.871344,-0.394161,0.668349,-0.963543,1.764171
2,1.141867,-0.840323,0.559127,-0.317048,0.381202,0.607865,-0.471583,0.538473,-0.009192,-0.163922,0.522896,0.675908,0.606399,-0.854470,0.026998,-0.758765,0.015678,0.494827,0.144748,0.043494
3,-3.237724,1.352510,-0.461185,-0.570249,1.174082,-2.032553,1.121010,0.902183,-0.077649,-0.839874,-0.400746,-0.048956,-0.075553,-0.176634,0.164792,0.432170,-0.022822,-0.220820,0.026663,-0.140462
4,0.133138,-1.746723,-0.016856,1.107963,0.131279,-0.170207,-0.414901,-0.580175,-0.064467,1.124889,0.069938,0.728251,-0.303892,-0.212234,0.155385,-0.665076,-0.055297,0.433666,0.599183,-0.357324


In [11]:
principalDf.reset_index(drop=True, inplace=True)

aux = prueba[['momento']]
aux.reset_index(drop=True, inplace=True)

finalDf = pd.concat([principalDf, aux], axis = 1)
finalDf.head()

,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,principal component 6,principal component 7,principal component 8,principal component 9,principal component 10,...,principal component 12,principal component 13,principal component 14,principal component 15,principal component 16,principal component 17,principal component 18,principal component 19,principal component 20,momento
0,-2.337290,-0.498829,-0.858843,-0.518616,-0.060498,-0.192489,0.085027,-0.087409,0.579366,-0.001253,...,0.273827,-0.064134,0.188470,0.165769,0.762987,-0.111273,-0.262081,0.053024,-0.253382,Lavadora
1,1.689775,-0.514729,-0.971589,-2.992900,1.103809,2.524404,0.129749,4.122140,1.998267,-0.305412,...,1.369755,-0.638422,-0.502953,-0.906961,-1.871344,-0.394161,0.668349,-0.963543,1.764171,Microondas
2,1.141867,-0.840323,0.559127,-0.317048,0.381202,0.607865,-0.471583,0.538473,-0.009192,-0.163922,...,0.675908,0.606399,-0.854470,0.026998,-0.758765,0.015678,0.494827,0.144748,0.043494,Aspiradora
3,-3.237724,1.352510,-0.461185,-0.570249,1.174082,-2.032553,1.121010,0.902183,-0.077649,-0.839874,...,-0.048956,-0.075553,-0.176634,0.164792,0.432170,-0.022822,-0.220820,0.026663,-0.140462,Timbre
4,0.133138,-1.746723,-0.016856,1.107963,0.131279,-0.170207,-0.414901,-0.580175,-0.064467,1.124889,...,0.728251,-0.303892,-0.212234,0.155385,-0.665076,-0.055297,0.433666,0.599183,-0.357324,Ducha


In [12]:
classes = ['Afeitadora', 'Aire_acondicionado', 'Alarma', 'Aspiradora', 'Cena', 'Ducha', 'Grifo', 'Lavadora', 
           'Licuadora', 'Microondas', 'Secador_pelo', 'Secadora', 'Silencio', 'Tecleo', 'Telefono', 'Timbre']

X_new = finalDf.loc[:, finalDf.columns != 'momento']
y_new = new_model.predict(X_new)
y_new2 = new_model.predict_proba(X_new)

reconocidos = 0
aciertos = 0
for i in range(len(X_new)):
    predicted_list = y_new[i].tolist()
    predicted_class = classes[y_new2[i].tolist().index(max(y_new2[i]))]
    predicted_probability = y_new2[i].tolist()[y_new2[i].tolist().index(max(y_new2[i]))]
    if max(y_new2[i]) > 0:
        if predicted_class == finalDf.momento[i]:
            #print(str(i) + '-  {}: {:.2f}% ----> {}'.format(predicted_class, predicted_probability*100, finalDf.momento[i]))
            aciertos += 1
            reconocidos += 1
        else:
            #print(str(i) + '-  {}: {:.2f}% ----> {}'.format(predicted_class, predicted_probability*100, finalDf.momento[i]))
            reconocidos += 1


print('Porcentaje de sonidos reconocidos: {:.2f}%.'.format(reconocidos/(len(X_new))*100))
print('Porcentaje de aciertos: {:.2f}%.'.format(aciertos/(len(X_new))*100))
print('Porcentaje de aciertos una vez reconoce el sonido: {:.2f}%.'.format(aciertos/reconocidos*100))

Porcentaje de sonidos reconocidos: 100.00%.
Porcentaje de aciertos: 7.92%.
Porcentaje de aciertos una vez reconoce el sonido: 7.92%.


# Conclusiones

Los resultados me sorprenden, dado que me esperaba que el modelo con el PCA me diese mejores resultados, dado que tenía un mejor score. Creo que podría deberse a que aunque son dos datasets de audios del mismo tipo, al ser sonidos diferentes, al aplicar el PCA de forma independiente me da unas relaciones y unos coeficientes diferentes.

Aparte de eso, el modelo es capaz de predecirme un 52% de los audios. Mi dataset consta de 4 tipos diferentes de sonidos para cada clase, por lo que no es un dataset muy rico, y mi dataset a predecir consta de 3 tipo diferentes para cada clase. Por lo que en base a esta escasa variedad de variables, los resultados considero que no son nada desdeñables.

Además, muchos de los sonidos que falla lo hace por gran similitud con otros, como el secador de pelo y la secadora, el grifo y la ducha, o la licuadora y la máquina de afeitar.

Como próximo paso, convendría enriquecer mi dataset.